In [2]:
%pip install xarray cfgrib eccodes -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cdsapi
c = cdsapi.Client()

Bounding Lateral/Longetudinal values for Swizerland

In [5]:
lat_min = 45.8
lat_max = 48.5
lon_min = 5.9
lon_max = 10.5

In [ ]:
import xarray as xr

# Open GRIB file using cfgrib engine
ds = xr.open_dataset("../../Data/Weather/Seasonal forecast anomalies on single levels.grib", engine="cfgrib")

# Subset to Switzerland only
swiss = ds['t2a'].sel(
    latitude=slice(lat_max, lat_min),  # North to South
    longitude=slice(lon_min, lon_max)  # West to East
)


/opt/homebrew/lib/python3.11/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


In [7]:
# Average over space (lat/lon) and ensemble members
monthly_avg = swiss.mean(dim=['latitude', 'longitude', 'number'])

# Loop over time dimension to print monthly forecast issue dates
for i in range(monthly_avg.time.size):
    issue_month = str(monthly_avg.time.values[i])[:10]  # e.g., '2021-01-01'
    
    # Mean over all lead times (or choose step=0 if you want only 1 month ahead)
    mean_forecast = monthly_avg.isel(time=i).mean(dim='step').values
    
    print(f"Issue Date: {issue_month} | Avg Temp Anomaly: {mean_forecast:.3f} K")


Issue Date: 2021-01-01 | Avg Temp Anomaly: 0.231 K
Issue Date: 2021-02-01 | Avg Temp Anomaly: 0.576 K
Issue Date: 2021-03-01 | Avg Temp Anomaly: 0.554 K
Issue Date: 2021-04-01 | Avg Temp Anomaly: 0.582 K
Issue Date: 2021-05-01 | Avg Temp Anomaly: 0.176 K
Issue Date: 2021-06-01 | Avg Temp Anomaly: 0.737 K
Issue Date: 2021-07-01 | Avg Temp Anomaly: 0.513 K
Issue Date: 2021-08-01 | Avg Temp Anomaly: 0.165 K
Issue Date: 2021-09-01 | Avg Temp Anomaly: 0.517 K
Issue Date: 2021-10-01 | Avg Temp Anomaly: 0.395 K
Issue Date: 2021-11-01 | Avg Temp Anomaly: 0.532 K
Issue Date: 2021-12-01 | Avg Temp Anomaly: 0.244 K
Issue Date: 2022-01-01 | Avg Temp Anomaly: 0.862 K
Issue Date: 2022-02-01 | Avg Temp Anomaly: 0.939 K
Issue Date: 2022-03-01 | Avg Temp Anomaly: 1.104 K
Issue Date: 2022-04-01 | Avg Temp Anomaly: 0.474 K
Issue Date: 2022-05-01 | Avg Temp Anomaly: 0.945 K
Issue Date: 2022-06-01 | Avg Temp Anomaly: 0.559 K
Issue Date: 2022-07-01 | Avg Temp Anomaly: 0.854 K
Issue Date: 2022-08-01 | Avg Te

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Average over space (lat/lon) and ensemble members
monthly_avg = swiss.mean(dim=['latitude', 'longitude', 'number'])

# Mean over steps (lead time) — gives monthly anomaly per issue date
monthly_mean_anomaly = monthly_avg.mean(dim='step')  # shape: (time,)

# Convert to pandas for easy plotting
dates = pd.to_datetime(monthly_mean_anomaly.time.values)
anomaly_values = monthly_mean_anomaly.values

# Plot
plt.figure(figsize=(10, 5))
plt.plot(dates, anomaly_values, marker='o')
plt.title("Monthly Avg Temperature Anomalies over Switzerland")
plt.xlabel("Forecast Issue Date")
plt.ylabel("Temp Anomaly (K)")
plt.grid(True)
plt.axhline(0, color='gray', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
